In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display, clear_output


In [ ]:
# Charger les données
df = pd.read_csv("Travel_details_dataset.csv")

# Nettoyage des coûts
def clean_cost(value):
    if isinstance(value, str):
        return float(''.join([c for c in value if c.isdigit() or c == '.']))
    return value

for col in ['Accommodation cost', 'Transportation cost']:
    df[col] = df[col].apply(clean_cost)

# Sélection des colonnes
columns_to_use = ['Destination', 'Duration (days)', 'Accommodation cost',
                  'Transportation cost', 'Accommodation type', 'Transportation type']
data = df[columns_to_use].copy()

# Encodage
label_encoders = {}
for col in ['Destination', 'Accommodation type', 'Transportation type']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Gestion des valeurs manquantes
for col in ['Duration (days)', 'Accommodation cost', 'Transportation cost']:
    data[col].fillna(data[col].mean(), inplace=True)

# Normalisation
features = ['Duration (days)', 'Accommodation cost', 'Transportation cost',
            'Accommodation type', 'Transportation type']
X = data[features].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Restauration des noms originaux de destination
data['Destination_nom'] = label_encoders['Destination'].inverse_transform(df['Destination'].astype('category').cat.codes)


FileNotFoundError: [Errno 2] No such file or directory: 'Travel details dataset 2.csv'

In [ ]:
# Interface interactive
duration_widget = widgets.IntSlider(value=5, min=1, max=15, step=1, description='Durée (jours):')
acc_cost_widget = widgets.IntSlider(value=800, min=100, max=3000, step=100, description='Coût hébergement:')
transp_cost_widget = widgets.IntSlider(value=300, min=50, max=1500, step=50, description='Coût transport:')
acc_type_widget = widgets.Dropdown(options=label_encoders['Accommodation type'].classes_.tolist(),
                                   description='Hébergement:')
transp_type_widget = widgets.Dropdown(options=label_encoders['Transportation type'].classes_.tolist(),
                                      description='Transport:')
button = widgets.Button(description="Recommander")
output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output()
        profil_utilisateur = {
            'Duration (days)': duration_widget.value,
            'Accommodation cost': acc_cost_widget.value,
            'Transportation cost': transp_cost_widget.value,
            'Accommodation type': label_encoders['Accommodation type'].transform([acc_type_widget.value])[0],
            'Transportation type': label_encoders['Transportation type'].transform([transp_type_widget.value])[0]
        }
        user_vect = scaler.transform([list(profil_utilisateur.values())])
        similarities = cosine_similarity(X_scaled, user_vect).flatten()
        data['similarité'] = similarities
        top_recos = data[['Destination_nom', 'Duration (days)', 'Accommodation cost',
                          'Transportation cost', 'similarité']].sort_values(by='similarité', ascending=False).head(5)
        display(top_recos)

button.on_click(on_button_click)
display(duration_widget, acc_cost_widget, transp_cost_widget, acc_type_widget, transp_type_widget, button, output)
